# Consolidating Employee Data

This is a project for Data Preparation where is used dataframes to read and merge employee data from different sources.

# Project description
Data comes in diffrenet shapes, sizes, and formats. The ability to merge and clean diffrent sou7rces for analysis is a fundamental skill for any data practitioner.

In this project is used format csv, excel and json format.

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile
try:
    with zipfile.ZipFile ("/content/drive/MyDrive/Colab Notebooks/datasets.zip", 'r') as zObj:
        zObj.extractall("/content/drive/MyDrive/Colab Notebooks")
except:
    print("Already Unzipped")


In [3]:
import pandas as pd

# Reading dataset

the following data in the `datasets` folder:
- __Office addresses__
    - Saved in `office_addresses.csv`.
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`.
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:

In [4]:
offices = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/datasets/office_addresses.csv")
df_offices_columns = offices.columns
print("Offices Address features:", df_offices_columns)
offices

Offices Address features: Index(['office', 'office_country', 'office_city', 'office_street',
       'office_street_number'],
      dtype='object')


,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


# read employee_information.xlsx

In [5]:
addresses = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/datasets/employee_information.xlsx')
df_addresses_columns = addresses.columns
print("employee information features:", df_addresses_columns)
addresses

employee information features: Index(['employee_id', 'employee_last_name', 'employee_first_name',
       'employee_country', 'employee_city', 'employee_street',
       'employee_street_number'],
      dtype='object')


,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7


# read 2 sheet de file employee_information and add a header because was lost

In [6]:
emergency_col_names = ['employee_id', 'last_name', 'first_name', 'emergency_contact', 'emergency_contact_number','relationship']

#usually header= None, this time has we don't have header was added the list above
emergency_contacts = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/datasets/employee_information.xlsx",
                                   sheet_name = "emergency_contacts",
                                   header = None,
                                   names = emergency_col_names)
df_emergency_contacts_columns = emergency_contacts.columns
print("\nEmergency Contact Information")
emergency_contacts,df_emergency_contacts_columns



Emergency Contact Information


(  employee_id last_name first_name emergency_contact emergency_contact_number  \
 0      A2R5H9    Hunman        Jax        Opie Hurst          +32-456-5556-84   
 1      H8K0L6      Siff       Tara   Wendy de Matteo         +44-020-5554-333   
 2      G4R7V0     Sagal      Gemma      John Newmark           +1-202-555-194   
 3      M1Z7U9    Coates        Tig       Venus Noone          +1-202-555-0130   
 
   relationship  
 0      Brother  
 1       Sister  
 2      Husband  
 3         Wife  ,
 Index(['employee_id', 'last_name', 'first_name', 'emergency_contact',
        'emergency_contact_number', 'relationship'],
       dtype='object'))

# read json file

In [7]:
employee_roles = pd.read_json('/content/drive/MyDrive/Colab Notebooks/datasets/employee_roles.json', orient='index')

#rename first column as employee_id
employee_roles = employee_roles.reset_index().rename(columns={'index':'employee_id'})

df_employee_roles_columns = employee_roles.columns

print("employee roles features:", df_employee_roles_columns)
employee_roles

employee roles features: Index(['employee_id', 'title', 'monthly_salary', 'team'], dtype='object')


,employee_id,title,monthly_salary,team
0,A2R5H9,CEO,$4500,Leadership
1,H8K0L6,CFO,$4500,Leadership
2,G4R7V0,Business Developer,$3000,Sales
3,M1Z7U9,Office Manager,$2000,People Operations


In [8]:
df_offices_columns, offices.shape

(Index(['office', 'office_country', 'office_city', 'office_street',
        'office_street_number'],
       dtype='object'),
 (3, 5))

In [9]:
df_addresses_columns,df_emergency_contacts_columns, addresses.shape, emergency_contacts.shape

(Index(['employee_id', 'employee_last_name', 'employee_first_name',
        'employee_country', 'employee_city', 'employee_street',
        'employee_street_number'],
       dtype='object'),
 Index(['employee_id', 'last_name', 'first_name', 'emergency_contact',
        'emergency_contact_number', 'relationship'],
       dtype='object'),
 (4, 7),
 (4, 6))

In [10]:
df_employee_roles_columns, employee_roles.shape

(Index(['employee_id', 'title', 'monthly_salary', 'team'], dtype='object'),
 (4, 4))

# Merge DF

In [11]:
# Merge adresses with offices
employees =  addresses.merge(offices, left_on='employee_country', right_on='office_country', how='left')
employees.shape

(4, 12)

In [12]:
#merge employees with roles
employees = employees.merge(employee_roles, left_on='employee_id', right_on='employee_id')
employees.shape

(4, 15)

In [13]:
#merge with emergency contacts
employees = employees.merge(emergency_contacts, on='employee_id')
employees.shape

(4, 20)

# check NaN and fill in if needed

In [14]:
#check nan
employees.isna().sum()

employee_id                 0
employee_last_name          0
employee_first_name         0
employee_country            0
employee_city               0
employee_street             0
employee_street_number      0
office                      1
office_country              1
office_city                 1
office_street               1
office_street_number        1
title                       0
monthly_salary              0
team                        0
last_name                   0
first_name                  0
emergency_contact           0
emergency_contact_number    0
relationship                0
dtype: int64

In [15]:
# Define the value to fill NaN with
fill_value = 'Remote'

#fill nan with 'Remote'
employees.fillna(fill_value, inplace=True)
employees

# # Fill null values in office columns
# for col in ["office", "office_country", "office_city", "office_street", "office_street_number"]:
#     employees[col].fillna("Remote", inplace=True)

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,office,office_country,office_city,office_street,office_street_number,title,monthly_salary,team,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,Leuven Office,BE,Leuven,Martelarenlaan,38.0,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,WeWork Office,GB,London,Old Street,207.0,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,ESB Office,US,New York City,Fifth Avenue,350.0,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Remote,Remote,Remote,Remote,Remote,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


# Check features in DF merged

In [16]:
#check duplicate features
employees.columns

Index(['employee_id', 'employee_last_name', 'employee_first_name',
       'employee_country', 'employee_city', 'employee_street',
       'employee_street_number', 'office', 'office_country', 'office_city',
       'office_street', 'office_street_number', 'title', 'monthly_salary',
       'team', 'last_name', 'first_name', 'emergency_contact',
       'emergency_contact_number', 'relationship'],
      dtype='object')

# subset Final Columns

In [17]:
#create final columns
final_columns = ['employee_id', 'employee_first_name', 'employee_last_name',
                 'employee_country', 'employee_city', 'employee_street', 'employee_street_number',
                 'emergency_contact', 'emergency_contact_number', 'relationship',
                 'monthly_salary', 'team', 'title', 'office', 'office_country',
                 'office_city', 'office_street', 'office_street_number']

#subset for final columns
employees_final = employees[final_columns]
employees_final

,employee_id,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote


In [18]:
#set employee_id as index
employees_final.set_index('employee_id', inplace=True)
employees_final

,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote
